In [18]:
# -------
# IMPORT LIBRAIRIES
# -------
import numpy as np
import pandas as pd
from tqdm import tqdm
from scipy.optimize import minimize
from python_module.pricing_model import BlackScholesModel, SABRModel

pd.options.display.float_format = '{:.2f}'.format

In [19]:
def compute_weights(constraints, vector, value):
    def objective_function(weights, vector, value):
        return (value - vector.dot(weights))**2
    x0 = [0] * len(vector.index)
    result = minimize(objective_function, x0, method='SLSQP',constraints=constraints, args=(vector, value))
    print(result)
    return result.x

In [20]:
# -------
# GENERATE MARKET DATA AND GREEKS
# -------

# Explicit pricing parameters
S0 = F = 5000
r = 0
option_type = 'put'
time_to_maturity = 5

# Market parameters
market_alpha = +0.15
market_beta  = +1.0
market_rho   = -0.9
market_nu    = 2.4

# Realized parameters
realized_alpha = +0.12
realized_beta  = +1.0
realized_rho   = -0.5
realized_nu    = 1.0
# Pre-processing
T = time_to_maturity / 250

# Pricing factory
market_data_list = list()
realized_data_list = list()

strike_delta_dict = {K: SABRModel.compute_option(F, K, T, market_alpha, market_beta, market_rho, market_nu, r, 'call')['delta'] for K in range(int(F*0.7), int(F*1.3))}
strike_delta_s = pd.Series(strike_delta_dict)
delta_strike_dict = {delta: (strike_delta_s-delta).abs().idxmin() for delta in [0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8]} 
strike_delta_s = pd.Series(delta_strike_dict).sort_values()
for K in strike_delta_s.values:

    option_type = 'call' if K > F else 'put'
    
    market_pricing_results = SABRModel.compute_option(F, K, T, market_alpha, market_beta, market_rho, market_nu, r, option_type, slide_list=[-0.05, 0.05])
    realized_pricing_results = SABRModel.compute_option(F, K, T, realized_alpha, realized_beta, realized_rho, realized_nu, r, option_type, slide_list=[-0.05, 0.05])
    
    market_data_list.append({
        'symbol': f"{time_to_maturity}T_{K}K_{option_type}",
        'option_type': option_type, 
        'time_to_maturity': time_to_maturity, 
        'S0':S0, 'r': r, 'F': F, 'K': K, 'T': T, **market_pricing_results})

    realized_data_list.append({
        'symbol': f"{time_to_maturity}T_{K}K_{option_type}",
        'option_type': option_type, 
        'time_to_maturity': time_to_maturity, 
        'S0':S0, 'r': r, 'F': F, 'K': K, 'T': T, **realized_pricing_results})

market_data_df = pd.DataFrame(market_data_list)
market_data_df = market_data_df.set_index('symbol')

realized_data_df = pd.DataFrame(realized_data_list)
realized_data_df = realized_data_df.set_index('symbol')

In [21]:
market_data_df

,option_type,time_to_maturity,S0,r,F,K,T,IV,price,delta,gamma,vega,theta,vanna,volga,slide pnl -0.05,slide pnl 0.05
symbol,,,,,,,,,,,,,,,,,
5T_4901K_put,put,5,5000,0,5000,4901,0.02,0.17,13.60,-0.20,0.00,1.98,-3.37,-1.34,799.59,100.18,36.38
5T_4942K_put,put,5,5000,0,5000,4942,0.02,0.16,22.15,-0.30,0.00,2.46,-3.98,-1.07,395.43,101.02,53.11
5T_4973K_put,put,5,5000,0,5000,4973,0.02,0.16,31.49,-0.40,0.00,2.73,-4.24,-0.59,106.73,95.42,68.09
5T_5001K_call,call,5,5000,0,5000,5001,0.02,0.15,41.60,0.50,0.00,2.82,-4.21,0.05,-0.04,85.37,82.29
5T_5027K_call,call,5,5000,0,5000,5027,0.02,0.14,28.54,0.40,0.00,2.73,-3.92,0.74,133.50,72.26,94.62
5T_5052K_call,call,5,5000,0,5000,5052,0.02,0.14,18.55,0.30,0.00,2.46,-3.41,1.36,499.87,57.33,104.05
5T_5080K_call,call,5,5000,0,5000,5080,0.02,0.13,10.35,0.20,0.00,1.98,-2.62,1.82,1083.87,39.91,109.59


In [22]:
# -------
# SLIDE & EXPECTED PNL
# -------
slide_df = market_data_df[list(filter(lambda x: x.startswith('slide'), market_data_df.columns))]
expected_pnl = realized_data_df['price']-market_data_df['price']
expected_pnl.name = 'expected_pnl'

In [25]:
constraints = [
    {'type': 'ineq', 'fun': lambda weights: slide_df.iloc[:, 1].dot(weights)},
    {'type': 'ineq', 'fun': lambda weights: slide_df.iloc[:, 0].dot(weights)},
    {'type': 'ineq', 'fun': lambda weights: expected_pnl.dot(weights)}
]
weights = compute_weights(constraints, slide_df.iloc[:, 0], value=1)

 message: Optimization terminated successfully
 success: True
  status: 0
     fun: 1.401315073519757e-13
       x: [ 4.271e-02 -2.321e-03 -2.108e-02 -2.121e-02 -8.927e-03
            8.054e-03  2.408e-02]
     nit: 2
     jac: [ 7.455e-05  7.643e-05  6.424e-05  4.468e-05  2.370e-05
            6.053e-06 -6.147e-06]
    nfev: 19
    njev: 2


In [33]:
slide_df.multiply(weights*10000, axis=0).sum()

slide pnl -0.05   10000.00
slide pnl 0.05     8820.26
dtype: float64

In [32]:
expected_pnl.multiply(weights*10000).sum()

6.0449806369433645e-06